In [1]:
import pandas as pd
from os import listdir
from collections import defaultdict, Counter
from tqdm import tqdm
import re

In [2]:
texttr_pos = listdir('./aclImdb/train/pos/')

In [3]:
texttr_neg = listdir('./aclImdb/train/neg/')

In [4]:
response = defaultdict(list)
v_dict = defaultdict(lambda : 0)

error_files = []
for filename in tqdm(texttr_pos):
    with open('./aclImdb/train/pos/' + filename) as file:
        try:
            content = " ".join(re.findall('\w+', file.read().lower()))
            for word in content.split():
                v_dict[word] += 1
            response['feedback'].append(content)
            response['flag'].append(True)
        except:
            error_files.append(filename)

for filename in tqdm(texttr_neg):
    with open('./aclImdb/train/neg/' + filename) as file:
        try:
            content = " ".join(re.findall('\w+', file.read().lower()))
            for word in content.split():
                v_dict[word] += 1
            response['feedback'].append(content)
            response['flag'].append(False)
        except:
            error_files.append(filename)

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:41<00:00, 298.84it/s]


In [11]:
dellwords = [i[0] for i in v_dict.items() if i[1] < 25]

In [12]:
data_train = pd.DataFrame(data=response)

In [13]:
len(data_train)

24994

In [14]:
uniq_word = set()
for dat in tqdm(data_train['feedback']):
    for word in dat.split():
        uniq_word.add(word)

100%|██████████████████████████████████████████████████████████████████████████| 24994/24994 [00:04<00:00, 5390.98it/s]


In [15]:
for word in dellwords:
    uniq_word.remove(word)

In [16]:
len(uniq_word)

11565

In [17]:
uniq_word_list = list(uniq_word)

In [18]:
word2index = {key:value for value, key in enumerate(uniq_word)}

In [19]:
l = len(uniq_word)
for  dat in tqdm(data_train['feedback']):
    counter = Counter(dat.split())
    row = [counter[uniq_word_list[i]] if uniq_word_list[i] in counter else 0 for i in range(l)]
    response['predate'].append(row)

100%|███████████████████████████████████████████████████████████████████████████| 24994/24994 [03:54<00:00, 106.73it/s]


In [20]:
data_train = pd.DataFrame(data=response)

In [21]:
data_train.to_csv('pain.csv')

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [50]:
x_train,x_test,y_train,y_test = train_test_split(data_train['predate'], data_train['flag'])

In [51]:
model = LogisticRegression()

In [52]:
y_train = [1 if i else 0 for i in y_train]

In [53]:
import numpy as np

In [54]:
x_train = np.array([np.array(i) for i in x_train])

In [55]:
x_train.shape

(18745, 11565)

In [56]:
model.fit(x_train, y_train, verbose=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [57]:
x_test = np.array([np.array(i) for i in x_test])

In [58]:
y_test = [1 if i else 0 for i in y_test]

In [59]:
model.score(x_test,y_test)

0.8724595935349656

In [62]:
import pickle

In [67]:
file = open('model', 'wb')
pickle.dump(model, file)
file.close()

In [79]:
dd = " ".join(re.findall('\w+', input('enter the message').lower()))
counter = Counter(dd.split())
row = [counter[uniq_word_list[i]] if uniq_word_list[i] in counter else 0 for i in range(l)]

if model.predict(np.array(row).reshape((1, -1)))[0]:
    print('pos')
else:
    print('neg')

enter the messageThis was a decent movie for the first half. Too many cheap BOO! moments but the tension builds, the bad guys are creepy and everything seems to be setting itself up nicely. The kids are not particularly deep but hey, that works for teens. <br /><br />Then it just gets ridiculous and tries way too hard- the "why in the world would he/they do that?" moments overwhelm anyone's capacity for suspension of disbelief, the twist involves too many ridiculous coincidences, and the title comes from a late attempt to philosophize some meaning into the film that goes nowhere and is quickly dropped. There was laughter in the theater at moments that were in no way supposed to be funny. <br /><br />Great premise but just badly written and doesn't hold together. Some very nice shots but they're hard to enjoy while you're rolling your eyes.
neg
